# Carregamento dos dados

In [59]:
import pandas as pd

# Definindo o caminho para o arquivo CSV com os dados das análises de aplicativos
file_path = '../Data/apps_reviews_validacao.xlsx - apps_reviews.csv'

# Carregando os dados do arquivo CSV em um DataFrame do pandas
# Usecols: Especifica as colunas a serem carregadas do arquivo
# 'reviewId', 'app_name', 'content', 'sentiment_polarity', 'sentiment' são as colunas selecionadas
# A última coluna nula é ignorada
comentarios = pd.read_csv(file_path, usecols=['reviewId', 'app_name', 'content', 'sentiment_polarity', 'sentiment'])

comentarios.loc[1665, 'content']

'Aplicativo chato vei toda hora aparece, agora quando eu tava escrevendo o anúncio apareceu 😠😡🤬🤬 aff já apareceu mais 2 vezes eu não posso fazer mais nada em meu celular que esse anúncio aparece 😠 tira logo do ar aff😡🤬🤬🤬'

# Pré-processamento

<p style="color:green">Converte emojis para texto</p>

In [60]:
import emoji

def replace_emojis(text):
    # Converte emojis no texto para suas representações textuais usando o pacote 'emoji'
    # 'language='pt'' especifica que a representação textual dos emojis deve estar em português
    return emoji.demojize(text, language='pt')

# Aplica a função 'replace_emojis' a cada elemento da coluna 'content' do DataFrame 'comentarios'
# Isso substituirá todos os emojis nos textos por suas representações textuais
comentarios['content'] = comentarios['content'].apply(replace_emojis)

# Exibe o conteúdo da linha 1665 após a substituição dos emojis
comentarios.loc[1665, 'content']

'Aplicativo chato vei toda hora aparece, agora quando eu tava escrevendo o anúncio apareceu :rosto_zangado::rosto_furioso::rosto_com_símbolos_na_boca::rosto_com_símbolos_na_boca: aff já apareceu mais 2 vezes eu não posso fazer mais nada em meu celular que esse anúncio aparece :rosto_zangado: tira logo do ar aff:rosto_furioso::rosto_com_símbolos_na_boca::rosto_com_símbolos_na_boca::rosto_com_símbolos_na_boca:'

<p style="color:green">Removendo todo e qualquer caracter que não seja latino</p>

In [61]:
import regex 

# Aplica uma substituição para remover caracteres não latinos do texto
# regex.sub é usado para substituir todos os caracteres que não são letras do alfabeto latino por um espaço
# O padrão '[^\p{Latin}]' identifica qualquer caractere que não seja uma letra latina
comentarios['content'] = comentarios['content'].apply(lambda x: regex.sub('[^\p{Latin}]', u' ', str(x)))

# Exibe o conteúdo da linha 1665 após a substituição dos caracteres não latinos
comentarios.loc[1665, 'content']

'Aplicativo chato vei toda hora aparece  agora quando eu tava escrevendo o anúncio apareceu  rosto zangado  rosto furioso  rosto com símbolos na boca  rosto com símbolos na boca  aff já apareceu mais   vezes eu não posso fazer mais nada em meu celular que esse anúncio aparece  rosto zangado  tira logo do ar aff rosto furioso  rosto com símbolos na boca  rosto com símbolos na boca  rosto com símbolos na boca '

<p style="color:green">Colocando todos os textos para minúsculo</p>

In [62]:
# Converte o texto na coluna 'content' para minúsculas
# A função lambda é usada para garantir que cada texto na coluna seja convertido para minúsculas
comentarios['content'] = comentarios['content'].apply(lambda x: str(x).lower())

# Exibe o conteúdo da linha 1665 após a conversão para minúsculas
comentarios.loc[1665, 'content']

'aplicativo chato vei toda hora aparece  agora quando eu tava escrevendo o anúncio apareceu  rosto zangado  rosto furioso  rosto com símbolos na boca  rosto com símbolos na boca  aff já apareceu mais   vezes eu não posso fazer mais nada em meu celular que esse anúncio aparece  rosto zangado  tira logo do ar aff rosto furioso  rosto com símbolos na boca  rosto com símbolos na boca  rosto com símbolos na boca '

<p style="color:green">Corrigindo alguns erros de ortografia</p>

In [63]:
comentarios.loc[1694, 'content']

'nao uzei so baixei por que travou a tela pedindo pra baixar nao concigui de outra forma se nao baixar'

In [64]:
from spellchecker import SpellChecker
spell = SpellChecker(language='pt')

def correct_spelling(text):
    corrected_text = []
    words = text.split()
    
    for word in words:
        # Corrige a ortografia de cada palavra usando o SpellChecker
        corrected_word = spell.correction(word)
        # Se 'corrected_word' for None, mantemos a palavra original
        corrected_text.append(corrected_word if corrected_word else word)
    
    # Junta as palavras corrigidas em uma única string
    return " ".join(corrected_text)

# Aplica a função 'correct_spelling' a cada elemento da coluna 'content' do DataFrame 'comentarios'
comentarios['content'] = comentarios['content'].apply(correct_spelling)

# Exibe o conteúdo da linha 1694 após a correção ortográfica
comentarios.loc[1694, 'content']

'não usei o baixei por que travou a tela pedindo para baixar não consigui de outra forma se não baixar'

<p style="color:green">Removendo stopwords</p>

In [85]:
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
nltk.download('stopwords')

def remove_stopwords(text):
    # Verifica se o input é uma string. Se não for, retorna uma string vazia.
    if not isinstance(text, str):
        return ""  
    
    # Obtém a lista de palavras de parada (stopwords) em português
    stops_list = stopwords.words("portuguese")
    # Adiciona a palavra "rosto" à lista de palavras de parada
    stops_list.append("rosto")
    # Remove a palavra "não" da lista de palavras de parada
    stops_list.remove("não")
    # Tokeniza o texto em palavras individuais
    word_tokens = word_tokenize(text)
    # Remove as palavras de parada da lista de tokens, convertendo palavras para minúsculas
    texto_sem_stops = [w for w in word_tokens if w.lower() not in stops_list] 
    # Junta as palavras restantes em uma única string
    return " ".join(texto_sem_stops)

# Aplica a função 'remove_stopwords' a cada elemento da coluna 'content' do DataFrame 'comentarios'
comentarios['content'] = comentarios['content'].apply(remove_stopwords)

# Exibe o conteúdo da linha 1694 após a remoção das palavras de parada
comentarios.loc[1694, 'content']

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\dougl\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


'não usei baixei travou tela pedindo baixar não consigui outra forma não baixar'

<p style="color:green">Lematização </p>

In [86]:
print(comentarios.loc[1433, 'content'])
print(comentarios.loc[76, 'content'])

ótimo amo demais agora áudios instaram silenciados todos destaques música nenhuma demorei tanto tempo escolher música colocar naqueles atores acontecer pena reclamando pé gosto demais desse jeito não dá certo
ótima opção compra única coisa chata tudo compra entrega individual ex compra lápis loja batom outra chega tudo individual ter lugar juntar todos pedidos mesma pessoa mandar todos juntos igual concorrência ajudava gente meio ambiente penso pó não dei estrelas


In [87]:
import spacy
nlp = spacy.load("pt_core_news_sm")

def lemmatize_text(text):
    # Processa o texto com o modelo spaCy carregado
    doc = nlp(text)
    # Obtém as lemas (formas canônicas) dos tokens, excluindo palavras de parada
    lemmatized_text = [token.lemma_ for token in doc if not token.is_stop]
    # Junta os lemas restantes em uma única string
    return " ".join(lemmatized_text)

# Aplica a função 'lemmatize_text' a cada elemento da coluna 'content' do DataFrame 'comentarios'
comentarios['content'] = comentarios['content'].apply(lemmatize_text)

In [88]:
print(comentarios.loc[1433, 'content'])
print(comentarios.loc[76, 'content'])

bom amo áudio instar silenciar destaques música demorei escolher música colocar em aquele ator acontecer pena reclamar pé gostar jeito certo
ótimo opção compra único chato compra entregar individual ex compra lápis loja batom chegar individual juntar pedido mesmo pessoa mandar junto igual concorrência ajudar gente ambiente pensar pó dar estrela


# Salva o Dataframe pré-processado

In [89]:
def salvar_csv(df, nome_arquivo):
    df.to_csv(nome_arquivo, index=False, encoding='utf-8')
    print(f"Arquivo salvo como {nome_arquivo}")

salvar_csv(comentarios, '../Data/content_pre_processado.csv')

Arquivo salvo como ../Data/content_pre_processado.csv
